# Sentiments

Calculate the Sentiments using twitter-XLM-roBERTa-base Model. [Model Card on Hugging Face](https://arxiv.org/pdf/2104.12250.pdf).[Corresponding Paper](https://arxiv.org/pdf/2104.12250.pdf).


In [1]:
from transformers import pipeline
import pandas as pd
from tqdm.auto import tqdm

tqdm.pandas()


model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

sentiment_task("Putin ist ein Kriegsverbrecher")

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/robinfeldmann/Library/Python/3.11/lib/python/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/robinfeldmann/Library/Python/3.11/lib/python/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


[{'label': 'negative', 'score': 0.9096924662590027}]

## Ukrainian Sentiments

In [49]:
df = pd.read_csv(
    "/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/Language_min_dedupl/ru.csv"
)

In [50]:
def calc_sentiments(text: str) -> dict[str, str | float]:
    sent = {"label": "error", "score": 0.0}
    try:
        sent = sentiment_task(text)[0]
    except Exception as e:
        print(e)
    return sent

In [51]:
df["sentiment"] = df["text"].progress_apply(calc_sentiments)

 34%|███▍      | 158372/461180 [2:01:18<3:50:18, 21.91it/s]

In [44]:
df_uk["sentiment_label"] = df_uk["sentiment"].apply(lambda x: x["label"])
df_uk["sentiment_score"] = df_uk["sentiment"].apply(lambda x: x["score"])

In [48]:
df_uk.to_csv("/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/Sentiments/uk.csv")